# Analysing the best performing model using Captum to understand the decision making process from the model

##### This notebook follows the official tutorial for BERT models from Captum as can be found in this link: https://captum.ai/tutorials/Bert_SQUAD_Interpret

In [1]:
import pandas as pd
import torch
from captum.attr import LayerIntegratedGradients, visualization as viz
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

# Prepare data and load model

In [2]:
# Load prediction and test data
model_pred_df = pd.read_csv(
    '../2. models/predictions/nickmuchi-sec-bert-finetuned-finance-classification_False_FinancialStrengthText_MAMConfig_2024-04-30_predictions.csv')
test_df = pd.read_csv('../1. data/final/test.csv')

# Identify the indices of texts with the three lowest and highest prediction values
lowest_indices = model_pred_df['1'].nsmallest(3).index
highest_indices = model_pred_df['1'].nlargest(3).index

# Retrieve the actual texts corresponding to these indices for analysis
lowest_texts = test_df.loc[lowest_indices, 'FinancialStrengthText']
highest_texts = test_df.loc[highest_indices, 'FinancialStrengthText']

In [3]:
# Set up the computation device for PyTorch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# Load the pre-trained model and its configuration from a checkpoint
checkpoint_path = '../results/final/llm/llm_fine-tine_MAMConfig/nickmuchi-sec-bert-finetuned-finance-classification/checkpoint-138'
config = AutoConfig.from_pretrained(checkpoint_path)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, config=config)
model.to(device)
model.eval()  # Set model to evaluation mode
model.zero_grad()  # Zero gradients to ready for new computation

In [5]:
# Load tokenizer to convert text into format suitable for the model
tokenizer = AutoTokenizer.from_pretrained('nickmuchi/sec-bert-finetuned-finance-classification')

# Define special token IDs that will be used to format input data properly
ref_token_id = tokenizer.pad_token_id  # Padding token for reference sequences
sep_token_id = tokenizer.sep_token_id  # Separator token (often used in BERT models)
cls_token_id = tokenizer.cls_token_id  # Classifier token (marks start of sequence in BERT)

# Captum part

#### Helper functions

In [6]:
def predict(inputs):
    """Make a forward pass through the model to obtain predictions."""
    return model(inputs)[0]

def construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id):
    """Prepare input and reference pairs required for Captum's attribution analysis."""
    text_ids = tokenizer.encode(text, add_special_tokens=False)
    input_ids = [cls_token_id] + text_ids + [sep_token_id]
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(text_ids) + [sep_token_id]
    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device)


def custom_forward(inputs):
    """Custom forward function to extract specific outputs for analysis with Captum."""
    preds = predict(inputs)
    return torch.softmax(preds, dim=1)[0][0].unsqueeze(-1)


def summarize_attributions(attributions):
    """Normalize and summarize attributions across the input tokens."""
    attributions = attributions.sum(dim=-1).squeeze(0)
    return attributions / torch.norm(attributions)

### Analyze the text

In [7]:
def analyze_texts(texts):
    """Analyze given texts to visualize model attributions and their impact on predictions."""
    lig = LayerIntegratedGradients(custom_forward, model.bert.embeddings)

    for text in texts:
        input_ids, ref_input_ids = construct_input_ref_pair(text, ref_token_id, sep_token_id, cls_token_id)
        score = predict(input_ids)

        attributions, delta = lig.attribute(inputs=input_ids, baselines=ref_input_ids, return_convergence_delta=True)
        attributions_sum = summarize_attributions(attributions)

        indices = input_ids[0].detach().tolist()
        all_tokens = tokenizer.convert_ids_to_tokens(indices)

        # Prepare data for visualization
        score_vis = viz.VisualizationDataRecord(
            attributions_sum,
            torch.softmax(score, dim=1)[0][0],
            torch.argmax(torch.softmax(score, dim=1)[0]),
            0,
            text,
            attributions_sum.sum(),
            all_tokens,
            delta)

        print('\033[1m', 'Visualization For Score', '\033[0m')
        viz.visualize_text([score_vis])

### Perform analysis on texts with the lowest and highest model predictions

In [8]:
print("Analyzing lowest texts:")
analyze_texts(lowest_texts)

Analyzing lowest texts:
 Visualization For Score 


 Visualization For Score 


 Visualization For Score 


In [9]:
print("Analyzing highest texts:")
analyze_texts(highest_texts)

Analyzing highest texts:
 Visualization For Score 


 Visualization For Score 


 Visualization For Score 


### Manually add and analyze a specific custom text

In [10]:
custom_text = [
    "As of June 2023, AMD has $6.3 billion in cash and cash equivalents against total debt of $2.5 billion. AMD took on debt to acquire Xilinx, but Xilinx generates healthy cash flow, and now that AMD has gained meaningful market share in PC and server CPUs, we are comfortable that AMD will generate healthy free cash flow and should be able to work down its debt obligations over time. AMD does not pay a dividend but has bought back shares in recent years as part of a share-repurchase program announced in Feb 2022the company has about $6 billion remaining as of June 2023. Wed expect any capital distributions in the years ahead to be done via additional buybacks as part of this program."]
analyze_texts(custom_text)

 Visualization For Score 
